# RA-Notch Interpretation Notebook

### Load Notch Results

In [166]:
notch_data = readRDS('/data/srlab/lrumker/MCSC_Project/notch/analysis_sc_tissue.rds')

cna_res = read.csv("/data/srlab/lrumker/MCSC_Project/notch/notch_cna_res.txt")

nampcs = read.csv("/data/srlab/lrumker/MCSC_Project/notch/notch_cna_NAM_PCs.txt")

In [167]:
gene_expr = notch_data[['exprs_norm']][,match(cna_res$cell_id,colnames(notch_data[['exprs_norm']]))]

In [168]:
# Confirm cell order matches
sum(cna_res$cell_id==colnames(gene_expr))==dim(gene_expr)[2]

[1] TRUE

In [169]:
# Confirm NAM PC1 and neighborhood correlations are perfectly correlated,
# since only one NAM PC included in the model
cor(cna_res$ncorrs, nampcs$PC1)

[1] -1

In [170]:
cor(cna_res$score_notch, nampcs$PC1)

[1] -0.5561183

In [171]:
cor(cna_res$score_notch, nampcs$PC2)

[1] -0.09916919

### Compute Gene Correlations to NAM-PC1

In [172]:
# Examine correlations only with variable genes
n_var_genes = 5000
gene_sds = apply(gene_expr, 1, FUN = sd)
gene_sds = gene_sds[order(gene_sds, decreasing = TRUE)]

In [173]:
var_genes = names(gene_sds)
var_gene_expr = gene_expr[match(var_genes, rownames(gene_expr)),]

In [174]:
# Scores are cell loadings on PC1
cna_scores = nampcs$PC1

In [175]:
corrs = cor(t(as.matrix(var_gene_expr)), cna_scores)
names(corrs) = var_genes

Warning message in cor(t(as.matrix(var_gene_expr)), cna_scores):
“the standard deviation is zero”


In [176]:
quanitfy_correlation <-function(sel_gene, expr_mat, ncorrs){
    my_expr = expr_mat[match(sel_gene, rownames(expr_mat)),]
    my_data = data.frame(ncorrs, my_expr)
    colnames(my_data) = c("corrs", "expr")
    new_summary = summary(lm(corrs ~ expr, data = my_data))
    new_p = new_summary$coefficients[2,4]
    new_coeff = new_summary$coefficients[2,1]
    new_corr = cor(my_expr, ncorrs)
    return(c(new_corr, new_p))
}

### Most highly correlated genes

In [177]:
corrs = corrs[order(corrs, decreasing = TRUE)]

In [178]:
corrs[1:5]

PRG4       FN1    CRTAC1     HTRA1    ERRFI1 
0.7891419 0.7191157 0.6689098 0.6572137 0.6340549

In [179]:
quanitfy_correlation("PRG4", var_gene_expr, cna_scores)

[1] 0.7891419 0.0000000

In [180]:
quanitfy_correlation("FN1", var_gene_expr, cna_scores)

[1] 0.7191157 0.0000000

### Load Genesets

In [181]:
rankList = corrs

In [182]:
library(fgsea)
library(GSA)

In [183]:
geneset_files = c("/data/srlab/lrumker/MSigDB_Sets/c2.all.v7.0.symbols.gmt.txt", #Immune
                 "/data/srlab/lrumker/MSigDB_Sets/c2.all.v7.0.symbols.gmt.txt") #Canonical

In [184]:
GeneSet_SizeBound_Upper = 500 # Maximum gene set size considered
GeneSet_SizeBound_Lower = 0 # Minimum gene set size considered

In [185]:
parseGeneSetGMT <-function(filepath, known_genes, sizeBound_Upper, sizeBound_Lower){
  Gene_Sets_DB = GSA.read.gmt(filepath)
  GeneSet_sizes = sapply(Gene_Sets_DB$genesets,length)
  recognized_genes = matrix(NA, nrow=length(GeneSet_sizes), ncol = max(GeneSet_sizes))
  for(i in c(1:length(GeneSet_sizes))){
    recognized_genes[i,c(1:GeneSet_sizes[i])] = Gene_Sets_DB$genesets[[i]]
  }
  recognized_genes = matrix(is.element(recognized_genes, known_genes), ncol = ncol(recognized_genes))
  GeneSet_sizes = apply(recognized_genes, 1, sum)
  retain_GeneSet = (GeneSet_sizes>=sizeBound_Lower)&(GeneSet_sizes<=sizeBound_Upper)
  Gene_Sets_DB$genesets = Gene_Sets_DB$genesets[retain_GeneSet]
  Gene_Sets_DB$geneset.names = Gene_Sets_DB$geneset.names[retain_GeneSet]
  Gene_Sets_DB$geneset.descriptions = Gene_Sets_DB$geneset.descriptions[retain_GeneSet]
  Gene_Sets_DB$geneset.sizes = GeneSet_sizes[retain_GeneSet]
  return(Gene_Sets_DB)
}

In [186]:
Gene_Sets_DB = parseGeneSetGMT(geneset_files[1], names(rankList), GeneSet_SizeBound_Upper,
                               GeneSet_SizeBound_Lower)
input_genesets = Gene_Sets_DB$genesets
names(input_genesets) = Gene_Sets_DB$geneset.names

...
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [187]:
# Confirm presence of some NOTCH genesets
names(input_genesets)[grep("NOTCH", names(input_genesets))]

[1] "KEGG_NOTCH_SIGNALING_PATHWAY"                                              
 [2] "PID_NOTCH_PATHWAY"                                                         
 [3] "REACTOME_ACTIVATED_NOTCH1_TRANSMITS_SIGNAL_TO_THE_NUCLEUS"                 
 [4] "REACTOME_SIGNALING_BY_NOTCH4"                                              
 [5] "REACTOME_SIGNALING_BY_NOTCH2"                                              
 [6] "REACTOME_PRE_NOTCH_EXPRESSION_AND_PROCESSING"                              
 [7] "REACTOME_NOTCH1_INTRACELLULAR_DOMAIN_REGULATES_TRANSCRIPTION"              
 [8] "REACTOME_PRE_NOTCH_PROCESSING_IN_GOLGI"                                    
 [9] "REACTOME_SIGNALING_BY_NOTCH1"                                              
[10] "REACTOME_SIGNALING_BY_NOTCH3"                                              
[11] "REACTOME_NOTCH_HLH_TRANSCRIPTION_PATHWAY"                                  
[12] "REACTOME_SIGNALING_BY_NOTCH"                                               
[13] "NGUYEN_NOTCH1_TARGETS_UP"                                                  
[14] "NGUYEN_NOTCH1_TARGETS_DN"                                                  
[15] "VILIMAS_NOTCH1_TARGETS_UP"                                                 
[16] "VILIMAS_NOTCH1_TARGETS_DN"                                                 
[17] "BIOCARTA_NOTCH_PATHWAY"                                                    
[18] "REACTOME_PRE_NOTCH_PROCESSING_IN_THE_ENDOPLASMIC_RETICULUM"                
[19] "REACTOME_NOTCH2_INTRACELLULAR_DOMAIN_REGULATES_TRANSCRIPTION"              
[20] "REACTOME_SIGNALING_BY_NOTCH1_IN_CANCER"                                    
[21] "REACTOME_FBXW7_MUTANTS_AND_NOTCH1_IN_CANCER"                               
[22] "REACTOME_SIGNALING_BY_NOTCH1_T_7_9_NOTCH1:M1580_K2555_TRANSLOCATION_MUTANT"
[23] "REACTOME_SIGNALING_BY_NOTCH1_HD_DOMAIN_MUTANTS_IN_CANCER"                  
[24] "REACTOME_NOTCH2_ACTIVATION_AND_TRANSMISSION_OF_SIGNAL_TO_THE_NUCLEUS"      
[25] "REACTOME_RUNX3_REGULATES_NOTCH_SIGNALING"                                  
[26] "REACTOME_NOTCH3_ACTIVATION_AND_TRANSMISSION_OF_SIGNAL_TO_THE_NUCLEUS"      
[27] "REACTOME_NOTCH3_INTRACELLULAR_DOMAIN_REGULATES_TRANSCRIPTION"              
[28] "REACTOME_NOTCH4_INTRACELLULAR_DOMAIN_REGULATES_TRANSCRIPTION"              
[29] "REACTOME_NOTCH4_ACTIVATION_AND_TRANSMISSION_OF_SIGNAL_TO_THE_NUCLEUS"      
[30] "REACTOME_NONCANONICAL_ACTIVATION_OF_NOTCH3"                                
[31] "REACTOME_NEGATIVE_REGULATION_OF_NOTCH4_SIGNALING"

### Conduct geneset enrichment analysis

In [188]:
FGSEA_output = fgsea(input_genesets, rankList, nperm=10000, 
                         minSize = 0, maxSize = 500, nproc = 0,
                         gseaParam = 1, BPPARAM = NULL)
FGSEA_output = FGSEA_output[order(FGSEA_output$padj),]

Warning message in fgsea(input_genesets, rankList, nperm = 10000, minSize = 0, maxSize = 500, :
“You are trying to run fgseaSimple. It is recommended to use fgseaMultilevel. To run fgseaMultilevel, you need to remove the nperm argument in the fgsea function call.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.38% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”


In [190]:
FGSEA_output_sig = FGSEA_output[which(FGSEA_output$padj<0.05),]
FGSEA_output_sig[grep("NOTCH", FGSEA_output_sig$pathway),c(1:3)]

pathway,pval,padj
<chr>,<dbl>,<dbl>
VILIMAS_NOTCH1_TARGETS_UP,0.0002883922,0.00742766
REACTOME_SIGNALING_BY_NOTCH,0.0020489333,0.02581080
NGUYEN_NOTCH1_TARGETS_DN,0.0052888527,0.04771678
REACTOME_NEGATIVE_REGULATION_OF_NOTCH4_SIGNALING,0.0054960541,0.04868755


## Characterizing NAM-PC2

### Compute Gene Correlations to NAM-PC1

In [191]:
# Scores are cell loadings on PC2
cna_scores = nampcs$PC2

In [192]:
corrs = cor(t(as.matrix(var_gene_expr)), cna_scores)
names(corrs) = var_genes

Warning message in cor(t(as.matrix(var_gene_expr)), cna_scores):
“the standard deviation is zero”


In [193]:
corrs = corrs[order(corrs, decreasing = TRUE)]

### Load Genesets

In [194]:
rankList = corrs

In [205]:
grep("HLA", names(corrs)[order(corrs)][1:100])

[1]  1  9 17 22 23 45 59 64 96

In [204]:
names(corrs)[order(corrs)][1:100][grep("HLA", names(corrs)[order(corrs)][1:100])]

[1] "HLA-B"    "HLA-A"    "HLA-C"    "HLA-E"    "HLA-DRA"  "HLA-F"    "HLA-DPB1"
[8] "HLA-DRB1" "HLA-DPA1"

In [195]:
Gene_Sets_DB = parseGeneSetGMT(geneset_files[2], names(rankList), GeneSet_SizeBound_Upper,
                               GeneSet_SizeBound_Lower)
input_genesets = Gene_Sets_DB$genesets
names(input_genesets) = Gene_Sets_DB$geneset.names

...
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [196]:
FGSEA_output = fgsea(input_genesets, rankList, nperm=10000, 
                         minSize = 0, maxSize = 500, nproc = 0,
                         gseaParam = 1, BPPARAM = NULL)
FGSEA_output = FGSEA_output[order(FGSEA_output$padj),]

Warning message in fgsea(input_genesets, rankList, nperm = 10000, minSize = 0, maxSize = 500, :
“You are trying to run fgseaSimple. It is recommended to use fgseaMultilevel. To run fgseaMultilevel, you need to remove the nperm argument in the fgsea function call.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.38% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”


In [197]:
FGSEA_output_sig_NAMPC2 = FGSEA_output[which(FGSEA_output$padj<0.05),]

In [199]:
head(nampcs)

,X,PC1,PC2
,<int>,<dbl>,<dbl>
1,0,2.823947e-03,0.0008072186
2,2,-8.987393e-04,0.0068240035
3,3,-4.635763e-03,0.0090568405
4,5,5.397257e-03,0.0082240497
5,6,-7.306052e-06,0.0140845026
6,7,-7.311816e-03,0.0058684357


In [198]:
FGSEA_output_sig_NAMPC2[grep("REACTOME", FGSEA_output_sig_NAMPC2$pathway),c(1:4)]

pathway,pval,padj,ES
<chr>,<dbl>,<dbl>,<dbl>
REACTOME_CROSS_PRESENTATION_OF_SOLUBLE_EXOGENOUS_ANTIGENS_ENDOSOMES,0.0002358491,0.003650091,-0.6241756
REACTOME_ANTIGEN_PROCESSING_CROSS_PRESENTATION,0.0002498126,0.003650091,-0.7000089
REACTOME_ENDOSOMAL_VACUOLAR_PATHWAY,0.0002248201,0.003650091,-0.9372824
REACTOME_ANTIVIRAL_MECHANISM_BY_IFN_STIMULATED_GENES,0.0002447381,0.003650091,-0.5853584
REACTOME_SIGNALING_BY_NOTCH4,0.0002466091,0.003650091,-0.5638637
REACTOME_DOWNSTREAM_SIGNALING_EVENTS_OF_B_CELL_RECEPTOR_BCR,0.0002447381,0.003650091,-0.5519878
REACTOME_ACTIVATION_OF_NF_KAPPAB_IN_B_CELLS,0.0002394063,0.003650091,-0.6186325
REACTOME_SIGNALING_BY_THE_B_CELL_RECEPTOR_BCR,0.0002545825,0.003650091,-0.5056536
REACTOME_SIGNALING_BY_NOTCH3,0.0002357934,0.003650091,-0.6423219


In [165]:
FGSEA_output_sig_NAMPC2[grep("REACTOME", FGSEA_output_sig_NAMPC2$pathway),c(1:4)]

pathway,pval,padj
<chr>,<dbl>,<dbl>
REACTOME_ANTIGEN_PROCESSING_CROSS_PRESENTATION,0.0002562132,0.003508898
REACTOME_ENDOSOMAL_VACUOLAR_PATHWAY,0.0002228164,0.003508898
REACTOME_IMMUNOREGULATORY_INTERACTIONS_BETWEEN_A_LYMPHOID_AND_A_NON_LYMPHOID_CELL,0.0002556891,0.003508898
REACTOME_ORC1_REMOVAL_FROM_CHROMATIN,0.0002445586,0.003508898
REACTOME_ANTIVIRAL_MECHANISM_BY_IFN_STIMULATED_GENES,0.0002498751,0.003508898
REACTOME_SIGNALING_BY_NOTCH4,0.0002506266,0.003508898
REACTOME_DOWNSTREAM_SIGNALING_EVENTS_OF_B_CELL_RECEPTOR_BCR,0.0002498751,0.003508898
REACTOME_ACTIVATION_OF_NF_KAPPAB_IN_B_CELLS,0.0002411963,0.003508898
REACTOME_SIGNALING_BY_THE_B_CELL_RECEPTOR_BCR,0.0002566076,0.003508898
